In [5]:
# Install required packages
!pip install torch torchvision ultralytics
!pip install opencv-python matplotlib
!pip install ipywidgets # for interactive displays

ERROR: Invalid requirement: '#'


In [4]:
# Uninstall existing torch packages
!pip uninstall torch torchvision torchaudio -y

# Install PyTorch with CUDA support (specific version)
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.0%2Bcu118-cp312-cp312-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.0%2Bcu118-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.0%2Bcu118-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.7.0%2Bcu118-cp312-cp312-win_amd64.whl (2908.4 MB)
Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.0%2Bcu118-cp312-cp312-win_amd64.whl (5.5 MB)
Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.0%2Bcu118-cp312-cp312-win_amd64.whl (4.1 MB)


In [27]:
# Check if NVIDIA GPU is available
!nvidia-smi

Mon May 19 13:01:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P5              5W /   48W |     303MiB /   8188MiB |     61%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
# Make sure ultralytics is installed properly
!pip install ultralytics

In [5]:
# Verify GPU setup and PyTorch CUDA capability
import subprocess
import sys
import os

print("=== Python Environment Info ===")
print(f"Python version: {sys.version}")

print("\n=== CUDA Environment Check ===")
try:
    nvidia_output = subprocess.check_output('nvidia-smi', shell=True, stderr=subprocess.STDOUT).decode()
    print("NVIDIA GPU detected!")
    print(nvidia_output)
except:
    print("No NVIDIA GPU detected or nvidia-smi not available.")
    print("If you have an NVIDIA GPU, install the latest drivers from: https://www.nvidia.com/download/index.aspx")

print("\n=== PyTorch CUDA Status ===")
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    # Test CUDA tensor creation
    x = torch.tensor([1.0, 2.0, 3.0]).cuda()
    print(f"Test tensor on CUDA: {x}")
else:
    print("PyTorch is not using CUDA. Possible reasons:")
    print("1. You don't have an NVIDIA GPU")
    print("2. NVIDIA drivers are not properly installed")
    print("3. CUDA toolkit is not installed or not compatible")
    print("4. PyTorch was installed without CUDA support")

=== Python Environment Info ===
Python version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]

=== CUDA Environment Check ===
NVIDIA GPU detected!
Mon May 19 13:17:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8              4W /   

In [6]:
# Import required libraries
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from ultralytics import YOLO
import time

# For display
from IPython.display import display, Image as IPImage
import ipywidgets as widgets

# Check if GPU is available
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.7.0+cu118
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4070 Laptop GPU


In [7]:
# Configure the model to work on available device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Note: Even without GPU, the model will still work on CPU, just slower

Using device: cuda


In [8]:
# Load a pretrained YOLO model (YOLOv8n by default)
# You can choose different model sizes: n (nano), s (small), m (medium), l (large), x (extra large)
model = YOLO('yolov8n.pt')  # this downloads the model if not already present

print(f"Model loaded: YOLOv8n")
print(f"Model supports the following classes: {model.names}")

Model loaded: YOLOv8n
Model supports the following classes: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse',

In [ ]:
# Load a larger YOLOv8 model with more classes
# Note: There are several models available that detect different sets of classes

# Option 1: YOLOv8x - The largest YOLOv8 model (still 80 COCO classes but higher accuracy)
model_large = YOLO('yolov8x.pt')
print(f"Large model loaded: YOLOv8x (highest accuracy)")
print(f"Number of classes: {len(model_large.names)}")

# Option 2: YOLOv8x-OBB - For oriented bounding boxes (still 80 COCO classes but can detect rotated objects)
try:
    model_obb = YOLO('yolov8x-obb.pt')
    print(f"OBB model loaded: YOLOv8x-OBB (oriented bounding boxes)")
    print(f"Number of classes: {len(model_obb.names)}")
except Exception as e:
    print(f"Could not load OBB model: {e}")

# Option 3: YOLOv8x-Pose - For human pose estimation (17 keypoints)
try:
    model_pose = YOLO('yolov8x-pose.pt')
    print(f"Pose model loaded: YOLOv8x-Pose (human pose detection)")
    print(f"Can detect human keypoints for detailed pose analysis")
except Exception as e:
    print(f"Could not load Pose model: {e}")

# Option 4: Segment model - For segmentation masks (80 COCO classes)
try:
    model_seg = YOLO('yolov8x-seg.pt')
    print(f"Segmentation model loaded: YOLOv8x-Seg (object segmentation)")
    print(f"Number of classes: {len(model_seg.names)}")
except Exception as e:
    print(f"Could not load Segmentation model: {e}")

# Display available classes in the main model
print("\nAvailable classes in the standard model:")
for idx, class_name in model.names.items():
    print(f"{idx}: {class_name}")

In [10]:
# Utility functions for object detection

def detect_objects_in_image(model, image_path, conf_threshold=0.25):
    """
    Detect objects in an image using the YOLO model
    
    Args:
        model: YOLO model
        image_path: Path to the image
        conf_threshold: Confidence threshold for detections
        
    Returns:
        results: YOLO results object
        img: Original image with bounding boxes
    """
    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Error: Image file {image_path} not found.")
        return None, None
    
    # Load the image directly with OpenCV first to check validity
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image {image_path}. The file may be corrupt or in an unsupported format.")
        return None, None
    
    # Run inference on the image
    results = model(img, conf=conf_threshold)
    
    # Get the annotated image with bounding boxes
    annotated_img = None
    for r in results:
        annotated_img = r.plot()
    
    if results and len(results) > 0:
        return results[0], annotated_img
    else:
        print("No detection results returned by the model.")
        return None, img

def display_detection_results(img, results):
    """
    Display detection results with annotations
    
    Args:
        img: Image with bounding boxes
        results: YOLO results object
    """
    # Check if image and results are valid
    if img is None or results is None:
        print("Error: Invalid image or detection results.")
        return
    
    # Convert from BGR (OpenCV default) to RGB for matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Plot the image with detections
    plt.figure(figsize=(12, 8))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title(f"Detected {len(results.boxes)} objects")
    plt.show()
    
    # Print detection details
    boxes = results.boxes
    if len(boxes) > 0:
        print(f"\nDetection Details:")
        print("-" * 50)
        for i, box in enumerate(boxes):
            cls_id = int(box.cls.item())
            class_name = results.names[cls_id]
            confidence = box.conf.item()
            x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0].tolist()]
            print(f"Object {i+1}: Class = {class_name}, Confidence = {confidence:.2f}, Bounding Box = [{x1}, {y1}, {x2}, {y2}]")
    else:
        print("No objects detected.")

In [11]:
# Test detection on a sample image from URL

# Download a test image
!curl -o test_image.jpg https://ultralytics.com/images/zidane.jpg

# Run detection
sample_path = "test_image.jpg"
results, annotated_img = detect_objects_in_image(model, sample_path)

# Display results
display_detection_results(annotated_img, results)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   166  100   166    0     0    485      0 --:--:-- --:--:-- --:--:--   489
100   166  100   166    0     0    484      0 --:--:-- --:--:-- --:--:--   489


AttributeError: 'NoneType' object has no attribute 'ndim'

In [12]:
# For uploading your own images (run this in Google Colab)
# If you're using a local Jupyter notebook, you can use other methods to select images

def upload_and_detect():
    """
    Function to upload an image and run object detection
    Note: This works best in Google Colab
    """
    try:
        from google.colab import files
        uploaded = files.upload()
        for filename in uploaded.keys():
            print(f"Uploaded file: {filename}")
            results, annotated_img = detect_objects_in_image(model, filename)
            display_detection_results(annotated_img, results)
    except ImportError:
        print("This function works best in Google Colab.")
        print("If you're running locally, please use the next cell to specify a path to your image.")

# Uncomment to use in Colab
# upload_and_detect()

In [13]:
# For detecting objects in local images

def detect_local_image(image_path, conf_threshold=0.25):
    """
    Run detection on a local image
    
    Args:
        image_path: Path to the local image
        conf_threshold: Confidence threshold
    """
    if not os.path.exists(image_path):
        print(f"Error: File {image_path} not found.")
        return
    
    print(f"Running detection on {image_path}...")
    results, annotated_img = detect_objects_in_image(model, image_path, conf_threshold)
    display_detection_results(annotated_img, results)
    
    return results, annotated_img

# Example usage (uncomment and change path to your image)
# image_path = "path/to/your/image.jpg"  # Change this to your image path
# detect_local_image(image_path)

In [14]:
# For processing video

def process_video(video_path, output_path='output_video.mp4', conf_threshold=0.25, save_video=True, display_frames=False):
    """
    Process a video file for object detection
    
    Args:
        video_path: Path to input video
        output_path: Path to save output video
        conf_threshold: Confidence threshold for detection
        save_video: Whether to save the processed video
        display_frames: Whether to display frames during processing (slows down execution)
    """
    if not os.path.exists(video_path):
        print(f"Error: Video file {video_path} not found.")
        return
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video properties: {width}x{height} at {fps} fps, {total_frames} frames")
    
    # Initialize video writer if saving
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Process the video
    frame_count = 0
    start_time = time.time()
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Process frame
            results = model(frame, conf=conf_threshold)
            annotated_frame = results[0].plot()
            
            # Save frame
            if save_video:
                out.write(annotated_frame)
            
            # Display frame
            if display_frames and frame_count % 5 == 0:  # Show every 5th frame
                img_rgb = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
                plt.figure(figsize=(10, 6))
                plt.imshow(img_rgb)
                plt.axis('off')
                plt.title(f"Frame {frame_count}/{total_frames}")
                plt.show()
            
            # Update progress
            frame_count += 1
            if frame_count % 50 == 0:
                elapsed = time.time() - start_time
                fps_processing = frame_count / elapsed
                estimated_total = elapsed * (total_frames / frame_count)
                remaining = estimated_total - elapsed
                print(f"Processed {frame_count}/{total_frames} frames ({frame_count/total_frames*100:.1f}%) - {fps_processing:.1f} fps - ETA: {remaining:.1f}s")
    
    except KeyboardInterrupt:
        print("Processing interrupted")
    
    finally:
        # Release resources
        cap.release()
        if save_video:
            out.release()
        
        print(f"Processing complete: {frame_count}/{total_frames} frames processed")
        if save_video and os.path.exists(output_path):
            print(f"Output saved to {output_path}")

# Example usage (uncomment and change path to your video)
# video_path = "path/to/your/video.mp4"  # Change this to your video path
# process_video(video_path, display_frames=False)

In [21]:
# Real-time object detection with webcam

def detect_webcam(camera_id=0, conf_threshold=0.25, save_video=False, output_path='webcam_output.mp4'):
    """
    Run real-time object detection using webcam
    
    Args:
        camera_id: Webcam ID (usually 0 for built-in webcam)
        conf_threshold: Confidence threshold
        save_video: Whether to save the output video
        output_path: Path to save output video if save_video is True
    """
    # Initialize webcam
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"Error: Could not open webcam with ID {camera_id}")
        return
    
    # Get webcam properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 30  # Target FPS
    
    # Initialize video writer if saving
    out = None
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print("Webcam object detection started. Press 'q' to quit.")
    
    try:
        while True:
            # Read frame
            ret, frame = cap.read()
            if not ret:
                break
            
            # Run detection
            results = model(frame, conf=conf_threshold)
            annotated_frame = results[0].plot()
            
            # Save frame if requested
            if save_video and out is not None:
                out.write(annotated_frame)
            
            # Display the frame
            cv2.imshow("YOLO Object Detection", annotated_frame)
            
            # Break the loop on 'q' press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    except KeyboardInterrupt:
        print("Webcam detection interrupted")
        
    finally:
        # Release resources
        cap.release()
        if save_video and out is not None:
            out.release()
        cv2.destroyAllWindows()
        print("Webcam detection stopped")
        if save_video:
            print(f"Output saved to {output_path}")

# Example usage (uncomment to run)
detect_webcam(camera_id=0)  # Use default webcam

Webcam object detection started. Press 'q' to quit.


0: 480x640 2 persons, 3 chairs, 2 tvs, 17.8ms
Speed: 2.1ms preprocess, 17.8ms inference, 5.6ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 2 persons, 3 chairs, 2 tvs, 17.8ms
Speed: 2.1ms preprocess, 17.8ms inference, 5.6ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 2 persons, 4 chairs, 2 tvs, 17.7ms
Speed: 2.4ms preprocess, 17.7ms inference, 6.1ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 2 persons, 4 chairs, 2 tvs, 17.7ms
Speed: 2.4ms preprocess, 17.7ms inference, 6.1ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 2 persons, 5 chairs, 2 tvs, 15.2ms
Speed: 1.8ms preprocess, 15.2ms inference, 5.5ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 2 persons, 5 chairs, 2 tvs, 15.2ms
Speed: 1.8ms preprocess, 15.2ms inference, 5.5ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 2 persons, 4 chairs, 2 tvs, 17.8ms
Speed: 2.3ms preprocess, 17.8ms infe

In [ ]:
# Use the model with a very low confidence threshold to detect more objects
def detect_webcam_max_classes(camera_id=0, conf_threshold=0.1, selected_model=None):
    """
    Run real-time object detection using webcam with very low confidence threshold
    to detect as many objects as possible
    
    Args:
        camera_id: Webcam ID (usually 0 for built-in webcam)
        conf_threshold: Very low confidence threshold to detect more objects
        selected_model: Which model to use (if None, uses the default model)
    """
    # Use the selected model or the default model
    detection_model = selected_model if selected_model is not None else model
    
    # Initialize webcam
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"Error: Could not open webcam with ID {camera_id}")
        return
    
    # Get webcam properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print("Webcam object detection started. Press 'q' to quit.")
    print(f"Using confidence threshold: {conf_threshold} (lower value = detect more objects)")
    print(f"Model can detect {len(detection_model.names)} classes")
    
    try:
        while True:
            # Read frame
            ret, frame = cap.read()
            if not ret:
                break
            
            # Run detection with lower confidence threshold to detect more objects
            results = detection_model(frame, conf=conf_threshold, verbose=False)
            
            # Draw detection results on the frame
            annotated_frame = results[0].plot()
            
            # Show info on frame
            cv2.putText(
                annotated_frame, 
                f"Objects: {len(results[0].boxes)} (Press 'q' to quit)", 
                (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (0, 255, 0), 
                2
            )
            
            # Display the frame
            cv2.imshow("YOLO Object Detection", annotated_frame)
            
            # Break the loop on 'q' press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    except KeyboardInterrupt:
        print("Webcam detection interrupted")
        
    finally:
        # Release resources
        cap.release()
        cv2.destroyAllWindows()
        print("Webcam detection stopped")

# Run this function to start detection with very low confidence threshold
detect_webcam_max_classes(conf_threshold=0.05)  # Uncomment to run

Webcam object detection started. Press 'q' to quit.
Using confidence threshold: 0.05 (lower value = detect more objects)
Model can detect 80 classes


In [ ]:
# For direct webcam detection with maximum class detection:
# This will run webcam detection with a very low confidence threshold (0.05)
# to detect as many objects as possible from the 80 COCO classes

# Uncomment the line below to start webcam detection
detect_webcam_max_classes(conf_threshold=0.05)  # Very low threshold to detect more objects

In [19]:
# Choose which model to use for detection with maximum sensitivity
def run_detection_with_selected_model():
    """Interactive widget to select model and run webcam detection"""
    import ipywidgets as widgets
    from IPython.display import display
    
    # Create widgets
    model_dropdown = widgets.Dropdown(
        options=[
            ('Standard YOLOv8n (80 classes)', 'standard'),
            ('Large YOLOv8x (80 classes, higher accuracy)', 'large'),
            ('Segmentation YOLOv8x-Seg (80 classes with masks)', 'seg'),
            ('Pose YOLOv8x-Pose (human pose keypoints)', 'pose'),
            ('OBB YOLOv8x-OBB (oriented bounding boxes)', 'obb'),
        ],
        value='standard',
        description='Model:',
    )
    
    confidence_slider = widgets.FloatSlider(
        value=0.1,
        min=0.01,
        max=0.5,
        step=0.01,
        description='Confidence:',
        tooltip='Lower values detect more objects with less certainty',
    )
    
    # Button to run detection
    run_button = widgets.Button(
        description='Run Detection',
        button_style='success',
        tooltip='Start webcam detection with selected model'
    )
    
    output = widgets.Output()
    
    # Define callback
    def on_button_click(b):
        with output:
            print(f"Starting detection with {model_dropdown.value} model and {confidence_slider.value} confidence threshold")
            
            # Select the appropriate model
            if model_dropdown.value == 'standard':
                selected_model = model
            elif model_dropdown.value == 'large':
                selected_model = model_large if 'model_large' in globals() else model
            elif model_dropdown.value == 'seg':
                selected_model = model_seg if 'model_seg' in globals() else model
            elif model_dropdown.value == 'pose':
                selected_model = model_pose if 'model_pose' in globals() else model
            elif model_dropdown.value == 'obb':
                selected_model = model_obb if 'model_obb' in globals() else model
            
            # Run detection with selected model
            detect_webcam_max_classes(
                conf_threshold=confidence_slider.value,
                selected_model=selected_model
            )
    
    # Connect the callback to the button
    run_button.on_click(on_button_click)
    
    # Display the UI
    display(widgets.VBox([
        widgets.HTML("<h3>Configure Detection Settings</h3>"),
        model_dropdown,
        confidence_slider,
        run_button,
        output
    ]))

# Uncomment to display the interactive selection widget
# run_detection_with_selected_model()

## Advanced: Custom Training

You can train YOLO on your own custom dataset to detect specific objects. Here are the basic steps:

1. Prepare a dataset in YOLO format
2. Create a dataset YAML configuration file
3. Train the model
4. Evaluate and export the model

The code cell below shows how to train a model on a custom dataset.

In [16]:
# Custom training on your own dataset
'''
# Example dataset.yaml file structure
# Save this as 'dataset.yaml'

path: /path/to/dataset  # dataset root directory
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')

nc: 3  # number of classes
names: ['person', 'car', 'bicycle']  # class names
'''

# Training function
def train_custom_model(yaml_path, epochs=100, batch_size=16, img_size=640):
    """
    Train a custom YOLO model
    
    Args:
        yaml_path: Path to dataset YAML file
        epochs: Number of training epochs
        batch_size: Batch size
        img_size: Input image size
    """
    # Create a new model from YOLO
    model = YOLO('yolov8n.pt')  # Start with pretrained model
    
    # Train the model
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        patience=50,  # Early stopping patience
        save=True,  # Save checkpoints
        device='0' if torch.cuda.is_available() else 'cpu'
    )
    
    print("Training complete!")
    return model

# Usage example
# yaml_path = "path/to/your/dataset.yaml"  # Change to your dataset YAML path
# trained_model = train_custom_model(yaml_path, epochs=50)

## Conclusion and Next Steps

Congratulations! You've now set up a complete YOLO object detection workflow. You can:

1. Detect objects in images
2. Process videos for object detection
3. Use your webcam for real-time detection
4. Train custom models for specific use cases

### Next Steps

- Try different YOLO models (yolov8s.pt, yolov8m.pt, etc.) for different speed/accuracy tradeoffs
- Experiment with confidence thresholds
- Create a custom dataset for your specific needs
- Deploy your model to edge devices or cloud services

For more information, visit the [Ultralytics YOLOv8 documentation](https://docs.ultralytics.com/).

In [22]:
!pip install carla-0.9.X-py3.7-win-amd64.egg


ERROR: Could not find a version that satisfies the requirement carla-0.9.X-py3.7-win-amd64.egg (from versions: none)
ERROR: No matching distribution found for carla-0.9.X-py3.7-win-amd64.egg
